In [1]:
from model import EncodeNetwork, LSTM_Network, DecodeNetwork
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

# Using Full Network Test

This is a test to make sure my model code works from file. This is right before I tested the whole pipeline with dataset.

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
print(device)

cuda


In [4]:
im = plt.imread('cat.png')
im = cv2.resize(im, (127,127), interpolation=cv2.INTER_AREA)
print(im[:,:,0])
im.transpose(2,0,1)
im = torch.tensor(np.rollaxis(im,2,start=0), device=device)
print(im[0])

[[0.13224629 0.14305346 0.15394774 ... 0.18562932 0.1788699  0.17661801]
 [0.15731683 0.16145258 0.16850427 ... 0.1879927  0.1855335  0.17637281]
 [0.17958714 0.17884216 0.19504492 ... 0.19050126 0.18888876 0.18222909]
 ...
 [0.41120997 0.4092585  0.41364428 ... 0.4006953  0.39346942 0.38103503]
 [0.40712425 0.40625086 0.41060445 ... 0.40380505 0.39608523 0.3868692 ]
 [0.39853546 0.40264794 0.40639818 ... 0.39880264 0.39327908 0.39192843]]
tensor([[0.1322, 0.1431, 0.1539,  ..., 0.1856, 0.1789, 0.1766],
        [0.1573, 0.1615, 0.1685,  ..., 0.1880, 0.1855, 0.1764],
        [0.1796, 0.1788, 0.1950,  ..., 0.1905, 0.1889, 0.1822],
        ...,
        [0.4112, 0.4093, 0.4136,  ..., 0.4007, 0.3935, 0.3810],
        [0.4071, 0.4063, 0.4106,  ..., 0.4038, 0.3961, 0.3869],
        [0.3985, 0.4026, 0.4064,  ..., 0.3988, 0.3933, 0.3919]],
       device='cuda:0')


In [5]:
enc = EncodeNetwork()
enc = enc.cuda()

lstm = LSTM_Network()
lstm = lstm.cuda()

dec = DecodeNetwork()
dec = dec.cuda()

In [6]:
encoded = enc(im).cuda()

In [7]:
h0 = torch.zeros((128, 4, 4, 4)).cuda()
s0 = torch.zeros((128, 4, 4, 4)).cuda()
s1,h1 = lstm(encoded,s0,h0)

In [8]:
print(s1.shape, h1.shape)
print(s1.device, h1.device)

torch.Size([128, 4, 4, 4]) torch.Size([128, 4, 4, 4])
cuda:0 cuda:0


In [9]:
h1_res = h1.reshape((1,)+h1.shape)
dec(h1_res).shape

torch.Size([1, 2, 32, 32, 32])

In [10]:
import time
start = time.time()
encoded = enc(im).cuda()
h0 = torch.zeros((128, 4, 4, 4)).cuda()
s0 = torch.zeros((128, 4, 4, 4)).cuda()
s1,h1 = lstm(encoded,s0,h0)
h1_res = h1.reshape((1,)+h1.shape)
dec(h1_res).shape
print(time.time() - start)

0.005682229995727539
